# Using Null

- teacher

id	| dept	| name	| phone	| mobile
----|-------|-------|-------|-----
101	| 1 | Shrivell	| 2753 | 07986 555 1234
102	| 1	| Throd	    | 2754 | 07122 555 1920
103	| 1	| Splint	| 2293	|
104 |	| Spiregrain | 3287	|
105 | 2	| Cutflower	 | 3212 | 07996 555 6574
106 |	| Deadyawn | 3345 |	
... |      |        |        |

- dept

id	| name
----|----
1	| Computing
2	| Design
3	| Engineering
... |

### Teachers and Departments
The school includes many departments. Most teachers work exclusively for a single department. Some teachers have no department.

[Selecting NULL values](https://sqlzoo.net/wiki/Selecting_NULL_values).

## 1. NULL, INNER JOIN, LEFT JOIN, RIGHT JOIN

List the teachers who have NULL for their department.

> _Why we cannot use =_   
> You might think that the phrase dept=NULL would work here but it doesn't - you can use the phrase dept IS NULL
> 
> _That's not a proper explanation._  
> No it's not, but you can read a better explanation at Wikipedia:NULL.

## 2.
Note the INNER JOIN misses the teachers with no department and the departments with no teacher.

## 3.
Use a different JOIN so that all teachers are listed.

## 4.
Use a different JOIN so that all departments are listed.

## 5. Using the [COALESCE](https://sqlzoo.net/wiki/COALESCE) function


Use COALESCE to print the mobile number. Use the number '07986 444 2266' if there is no number given. **Show teacher name and mobile number or '07986 444 2266'**

## 6.
Use the COALESCE function and a LEFT JOIN to print the teacher name and department name. Use the string 'None' where there is no department.

## 7.
Use COUNT to show the number of teachers and the number of mobile phones.

## 8.
Use COUNT and GROUP BY **dept.name** to show each department and the number of staff. Use a RIGHT JOIN to ensure that the Engineering department is listed.

## 9. Using [CASE](https://sqlzoo.net/wiki/CASE)


Use CASE to show the **name** of each teacher followed by 'Sci' if the teacher is in **dept** 1 or 2 and 'Art' otherwise.

## 10.
Use CASE to show the name of each teacher followed by 'Sci' if the teacher is in dept 1 or 2, show 'Art' if the teacher's dept is 3 and 'None' otherwise.